In [1]:
!pip install numpy

In [2]:
import numpy as np

## Création des intéractions

In [35]:
Reply = np.array([
    [1, 5, 5, 0, 3],
    [2, 0, 5, 0, 3],
    [0, 0, 0, 0, 0],
    [2, 0, 0, 4, 0],
    [0, 4, 2, 0, 0]
])

Retweet = np.array([
    [0, 2, 4, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 4, 3],
    [0, 2, 5, 0, 0]
])

Like = np.array([
    [2, 5, 1, 0, 3],
    [0, 0, 0, 0, 0],
    [0, 5, 0, 3, 3],
    [1, 0, 0, 4, 1],
    [0, 0, 9, 0, 0]
])

## Statistiques d’engagement

In [4]:
# 1. L'utilisateur qui répond plus que les autres
A = Reply.sum(axis=1)
np.argmax(A)

0

In [5]:
# 2. L'utilisateur qui retweet plus que les autres
B = Retweet.sum(axis=1)
np.argmax(B)
# Affiche que le premier, problème quand plusieurs

3

In [6]:
# 3. Les utilisateurs qui n'ont pas de retweets et pas de replies
C = A + B
np.argwhere(C == 0)

array([[2]], dtype=int64)

In [7]:
# 4. Le nombre de like de chaque utilisateur à ses propres tweets
Id = np.eye(5)
D = Like * Id
D = D.sum(axis=0)
D

array([2., 0., 0., 4., 0.])

In [8]:
# 5. Le nombre des Likes que chaque utilisateur a donné aux tweets autre que ces propres tweets
H = Like - np.diag(D)
E = H.sum(axis=1)
E

array([ 9.,  0., 11.,  2.,  9.])

In [9]:
# 6. les utilisateurs qui aiment leurs propres tweets plus qu’ils aiment les tweets des autres
F = D > E
F.astype(int)
F = np.argwhere(F == 1)
F

array([[3]], dtype=int64)

In [10]:
# 7. Leur nombre
F.shape[1]

1

In [11]:
# 8. Le nombre des interactions que chaque utilisateur a fait
G = np.concatenate((Reply, Retweet, Like), axis=1)
G.sum(axis=1)

array([31, 10, 11, 19, 22])

## Statistiques sur les interactions

In [12]:
# 1. Le max des Likes que chaque utilisateur a reçu (des utilisateurs autres que lui-même)
H, H.sum(axis=0), H.max(axis=0)

(array([[0., 5., 1., 0., 3.],
        [0., 0., 0., 0., 0.],
        [0., 5., 0., 3., 3.],
        [1., 0., 0., 0., 1.],
        [0., 0., 9., 0., 0.]]),
 array([ 1., 10., 10.,  3.,  7.]),
 array([1., 5., 9., 3., 3.]))

In [13]:
# 2. Les utilisateurs qui ont reçu des Likes pour tous leurs tweets au moins par une personne autre qu’elle
Tweets = np.array([2, 5, 10, 4, 3])
I = H == Tweets

np.argwhere(H == Tweets)

array([[0, 1],
       [0, 4],
       [2, 1],
       [2, 4]], dtype=int64)

In [14]:
# 3. Le nombre des arcs entrants (ayant au moins une interaction)
SumTotal = Reply + Like + Retweet
Sum = (SumTotal > 0).astype(int)
SumEntrants = Sum.sum(axis=0)
SumEntrants, SumEntrants.sum()

(array([3, 3, 3, 2, 4]), 15)

In [15]:
# 4. Le nombre des arcs sortants (qui ont fait au moins une interaction)
SumSortants = Sum.sum(axis=1)
SumSortants, SumSortants.sum()

(array([4, 3, 3, 3, 2]), 15)

In [16]:
# 5. Les nœuds où le nombre des arcs entrants est égale au nombre de ceux sortants
np.argwhere(SumEntrants == SumSortants)

array([[1],
       [2]], dtype=int64)

In [17]:
# 6. Les Likes qu’ils ont fait entre eux
indexes = np.argwhere(SumEntrants == SumSortants)
Like[indexes].T[indexes].sum(axis=0)

array([[[0, 5]]])

## Popularité 

In [20]:
Like = Like - np.diag(D)
Reply = Reply - np.diag((Reply * np.eye(5)).sum(axis=0))
Like, Reply

(array([[0., 5., 1., 0., 3.],
        [0., 0., 0., 0., 0.],
        [0., 5., 0., 3., 3.],
        [1., 0., 0., 0., 1.],
        [0., 0., 9., 0., 0.]]),
 array([[0., 5., 5., 0., 3.],
        [2., 0., 5., 0., 3.],
        [0., 0., 0., 0., 0.],
        [2., 0., 0., 0., 0.],
        [0., 4., 2., 0., 0.]]))

In [36]:
# Score Reply
Reply = np.where((Reply > Like), Reply*-0.5, Reply)
Like = np.where((Reply == Like), Like*-0.25, Like)
Like, Reply

(array([[ 2.  , -1.25,  1.  , -0.  , -0.75],
        [ 0.  , -0.  ,  0.  , -0.  ,  0.  ],
        [-0.  ,  5.  , -0.  ,  3.  ,  3.  ],
        [ 1.  , -0.  , -0.  , -1.  ,  1.  ],
        [-0.  ,  0.  ,  9.  , -0.  , -0.  ]]),
 array([[ 1. ,  5. , -2.5,  0. ,  3. ],
        [-1. ,  0. , -2.5,  0. , -1.5],
        [ 0. ,  0. ,  0. ,  0. ,  0. ],
        [-1. ,  0. ,  0. ,  4. ,  0. ],
        [ 0. , -2. ,  2. ,  0. ,  0. ]]))

In [37]:
# Score Retweet
Diag = np.diag((Retweet * np.eye(5)).sum(axis=0))
Retweet = Retweet - Diag + np.where((Diag > 0), Diag * -0.25, Diag)
Retweet

array([[ 0.,  2.,  4.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  3.],
       [ 0.,  2.,  5.,  0.,  0.]])

In [40]:
# Score Likes
TweetsNum = Tweets.sum()/2
Like = np.where((Like > TweetsNum), Like*1.5, Like)
Like

array([[ 2.  , -1.25,  1.  , -0.  , -0.75],
       [ 0.  , -0.  ,  0.  , -0.  ,  0.  ],
       [-0.  ,  5.  , -0.  ,  3.  ,  3.  ],
       [ 1.  , -0.  , -0.  , -1.  ,  1.  ],
       [-0.  ,  0.  ,  9.  , -0.  , -0.  ]])

In [42]:
# Score de popularité
Score = Like + Retweet + Reply
Score.mean()

1.66